In [1]:
#import Library
from keras.models import load_model  
from PIL import Image, ImageOps 
import numpy as np
import os
from tensorflow.keras.layers import DepthwiseConv2D



In [3]:
# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

In [5]:
# Custom DepthwiseConv2D to handle 'groups' argument issue
class CustomDepthwiseConv2D(DepthwiseConv2D):
    def __init__(self, *args, **kwargs):
        if 'groups' in kwargs:
            del kwargs['groups']  
        super(CustomDepthwiseConv2D, self).__init__(*args, **kwargs)

In [7]:
# Load the model with custom objects
custom_objects = {'DepthwiseConv2D': CustomDepthwiseConv2D}
model = load_model("keras_model.h5", custom_objects=custom_objects)

# Load the labels
class_names = open("labels.txt", "r").readlines()



def predict_image(image_path):
    # Create the array of the right shape to feed into the keras model
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

    # Load the image
    image = Image.open(image_path).convert("RGB")

    # Resize the image to 224x224
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

    # Convert image to numpy array
    image_array = np.asarray(image)

    # Normalize the image
    normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

    # Load the image into the array
    data[0] = normalized_image_array

    # Predict the model
    prediction = model.predict(data)
    index = np.argmax(prediction)
    class_name = class_names[index]
    confidence_score = prediction[0][index]

    # Print prediction and confidence score
    print("Class:", class_name[2:], end="")  # Removing any leading characters (e.g., whitespace or '0')
    print("Confidence Score:", confidence_score)

if __name__ == "__main__":
    # Ask user for the image file path
    image_path = input("Please enter the path to the image file: ").strip()

    # Normalize path slashes
    image_path = image_path.replace("\\", "/")

    # Check if the file exists
    if os.path.isfile(image_path):
        predict_image(image_path)
    else:
        print(f"Error: File does not exist at {image_path}. Please check the path.")



C:\Users\ratan\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Please enter the path to the image file:  C:\Users\ratan\Desktop\Dataset\KneeXrayData\ClsKLData\Project\kneeKL224\test\1\9004184L.png


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Class: Moderate
Confidence Score: 0.9480606
